In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
# web scrapping
import bs4 as bs
import requests
import lxml
from functools import reduce
# matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from ipysigma import Sigma
from pyvis.network import Network
import requests
from bs4 import BeautifulSoup
from io import StringIO
from dbconnection import MySQLDatabase
from utils import getSymbols, getData, get_last_date, get_marketid_simbols
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="pandas")
sns.set_theme()

In [2]:
db = MySQLDatabase("financialmarkets")

## Se obtienen simbolos de wikipedia

In [7]:
tab_info = getSymbols('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
tab_info 

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [ ]:
date = datetime.today()
today = date.strftime('%Y-%m-%d')
today


In [5]:
symbols = tab_info['Symbol'].to_list()

In [ ]:
print(yf.__version__)

In [ ]:
# Example tickers (use full S&P 500 list if you want)
# tickers = stocks2
# selección de muestra
stocks2= symbols
start_date = '2020-01-01'
today = datetime.today() - timedelta(days=0)
end_date = today.strftime("%Y-%m-%d")
# data frame precios de cierre
data = getData(stocks2, start_date, end_date)
data.head()


In [ ]:
# ---------------------------
# 3️⃣ Insertar/actualizar mercados
# ---------------------------
markets = pd.DataFrame({
    'market_name': ['S&P 500'],
    'country': ['USA'],
    'currency': ['USD']
})
markets

In [ ]:
db.insert_to_db(markets, tabla="markets", batch_size=5000)

In [ ]:
# Obtener market_id
market_id = db.execute_query("SELECT * FROM markets")
market_id

**Tabla 2: sectores**

In [13]:
sectors_unique = tab_info[['GICS Sector', 'GICS Sub-Industry']].drop_duplicates()
sectors_unique.columns = ['sector_name', 'sub_industry']

In [ ]:
sectors_unique

In [ ]:
db.insert_to_db(sectors_unique, tabla="sectors", batch_size=5000)

In [ ]:
sector_map = db.execute_query("SELECT * FROM sectors")


In [ ]:
sector_map

**Empresas**

In [ ]:

# Agregar sector_id
df = tab_info.merge(sector_map, left_on=['GICS Sector','GICS Sub-Industry'], right_on=['sector_name','sub_industry'], how='left')

companies = df[['Symbol','Security','sector_id','Headquarters Location','CIK','Founded']]
companies.columns = ['symbol','name','sector_id','headquarters','cik','founded']
companies['founded'] = [str(x)[:4] for x in companies['founded']]
companies = companies.reset_index(drop=True)

In [ ]:
companies

In [ ]:
db.insert_to_db(companies, tabla="companies", batch_size=100)

In [ ]:
data = data.dropna(axis=1)


In [ ]:
# Obtener mapping symbol -> company_id
company_map = db.execute_query("SELECT company_id, symbol FROM companies")
company_map

**Empresas**

In [ ]:
company_map.columns

In [ ]:
market_companies_df = tab_info[['Symbol','Date added']].merge(company_map, left_on='Symbol', right_on='symbol', how='left')
market_companies_df['market_id'] = [market_id['market_id'][0] for x in market_companies_df['Symbol']]
market_companies_df = market_companies_df[['market_id','company_id','Date added']]
market_companies_df.columns = ['market_id','company_id','date_added']
market_companies_df


In [ ]:
db.insert_to_db(market_companies_df, tabla="market_companies", batch_size=100)

## Ingresar precios

In [6]:
# mercado
mercados = ['S&P 500']
# dataframe con compañias y id
company_map = db.execute_query("SELECT company_id, symbol FROM companies")
for mercado in mercados: 
    market_id, list_symbols = get_marketid_simbols(mercado)
    
    for symbol in list_symbols:
        start_date = get_last_date(symbol)
        print(f"Descargando {symbol} desde {start_date}")
        
        data = yf.download(symbol, start=start_date, progress=False, auto_adjust=False)
        
        if data.empty:
            continue
        
        data = data.reset_index()
        data.columns = [x for x,y in data.columns]
        #data['Symbol'] = [symbol for x in data['Symbol']]
        data = data.rename(columns={
            'Date': 'date',
            'Open': 'open_price',
            'Close': 'close_price',
            'High': 'high_price',
            'Low': 'low_price',
            'Volume': 'volume'
        })
        
        data['date'] = pd.to_datetime(data['date'], "%Y-%m-%d")
        # Interpolación lineal
    
        data = data.set_index('date').asfreq('D')
        #print(data.reset_index())
        
        data[['open_price','close_price','high_price','low_price','volume']] = \
            data[['open_price','close_price','high_price','low_price','volume']].interpolate(method='linear')
        data.loc[:,'symbol'] = [symbol for x in data['open_price']]
        data = data.reset_index()
        data = data.merge(company_map, left_on='symbol', right_on='symbol', how='left')
        data.loc[:,'market_id'] = [market_id for x in data['open_price']]
        # limpieza
        data_final = data[['company_id','market_id','date','open_price','close_price',
                            'high_price','low_price','volume']]
        data_final.loc[:,'company_id'] = data_final['company_id'].astype(int)
        data_final.loc[:,'market_id'] = data_final['market_id'].astype(int)
        data_final.loc[:,'date'] = pd.to_datetime(data_final['date'])
        data_final.loc[:,'open_price'] = data_final['open_price'].astype(float)
        data_final.loc[:,'close_price'] = data_final['close_price'].astype(float)
        data_final.loc[:,'high_price'] = data_final['high_price'].astype(float)
        data_final.loc[:,'low_price'] = data_final['low_price'].astype(float)
        data_final.loc[:,'volume'] = data_final['volume'].astype(int)
        # inserta datos a base 
        db.insert_to_db(data_final, tabla="stock_prices", batch_size=500)



Descargando MMM desde 2025-10-18
Descargando AOS desde 2025-10-18
Descargando ABT desde 2025-10-18
Descargando ABBV desde 2025-10-18
Descargando ACN desde 2025-10-18
Descargando ADBE desde 2025-10-18
Descargando AMD desde 2025-10-18
Descargando AES desde 2025-10-18
Descargando AFL desde 2025-10-18
Descargando A desde 2025-10-18
Descargando APD desde 2025-10-18
Descargando ABNB desde 2025-10-18
Descargando AKAM desde 2025-10-18
Descargando ALB desde 2025-10-18
Descargando ARE desde 2025-10-18
Descargando ALGN desde 2025-10-18
Descargando ALLE desde 2025-10-18
Descargando LNT desde 2025-10-18
Descargando ALL desde 2025-10-18
Descargando GOOGL desde 2025-10-18
Descargando GOOG desde 2025-10-18
Descargando MO desde 2025-10-18
Descargando AMZN desde 2025-10-18
Descargando AMCR desde 2025-10-18
Descargando AEE desde 2025-10-18
Descargando AEP desde 2025-10-18
Descargando AXP desde 2025-10-18
Descargando AIG desde 2025-10-18
Descargando AMT desde 2025-10-18
Descargando AWK desde 2025-10-18
De


1 Failed download:
['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


Datos vacios, {symbol} actualizado hasta última fecha disponible
Descargando BBY desde 2025-10-18
Descargando TECH desde 2025-10-18
Descargando BIIB desde 2025-10-18
Descargando BLK desde 2025-10-18
Descargando BX desde 2025-10-18
Descargando XYZ desde 2025-10-18
Descargando BK desde 2025-10-18
Descargando BA desde 2025-10-18
Descargando BKNG desde 2025-10-18
Descargando BSX desde 2025-10-18
Descargando BMY desde 2025-10-18
Descargando AVGO desde 2025-10-18
Descargando BR desde 2025-10-18
Descargando BRO desde 2025-10-18



1 Failed download:
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-01-01 -> 2025-10-19)')


Descargando BF.B desde 2000-01-01
Datos vacios, {symbol} actualizado hasta última fecha disponible
Descargando BLDR desde 2025-10-18
Descargando BG desde 2025-10-18
Descargando BXP desde 2025-10-18
Descargando CHRW desde 2025-10-18
Descargando CDNS desde 2025-10-18
Descargando CPT desde 2025-10-18
Descargando CPB desde 2025-10-18
Descargando COF desde 2025-10-18
Descargando CAH desde 2025-10-18
Descargando KMX desde 2025-10-18
Descargando CCL desde 2025-10-18
Descargando CARR desde 2025-10-18
Descargando CAT desde 2025-10-18
Descargando CBOE desde 2025-10-18
Descargando CBRE desde 2025-10-18
Descargando CDW desde 2025-10-18
Descargando COR desde 2025-10-18
Descargando CNC desde 2025-10-18
Descargando CNP desde 2025-10-18
Descargando CF desde 2025-10-18
Descargando CRL desde 2025-10-18
Descargando SCHW desde 2025-10-18
Descargando CHTR desde 2025-10-18
Descargando CVX desde 2025-10-18
Descargando CMG desde 2025-10-18
Descargando CB desde 2025-10-18
Descargando CHD desde 2025-10-18
Desca

In [ ]:
# Example tickers (use full S&P 500 list if you want)
#tickers = stocks2

# Download adjusted close prices
#prices = yf.download(tickers, start='2020-01-01', end='2025-01-01')['Close']

#prices = prices.fillna(method='bfill')
# Calculate daily returns
returns = data.pct_change().dropna()

# Compute correlation matrix
correlation_matrix = returns.corr()

# Set correlation threshold
threshold = 0.7

# Create graph
G = nx.Graph()

# Add nodes
for ticker in correlation_matrix.columns:
    G.add_node(ticker)

# Add edges with correlation above threshold
for i, stock1 in enumerate(correlation_matrix.columns):
    for j, stock2 in enumerate(correlation_matrix.columns):
        if i < j:
            corr = correlation_matrix.iloc[i, j]
            if abs(corr) >= threshold:
                G.add_edge(stock1, stock2, weight=corr)

# Display with ipysigma
#Sigma(G)
# Initialize pyvis network
#net = Network(notebook=True)
#net.from_nx(G)

# Save and show the graph
#net.show("graph.html")
Sigma.write_html(
    G,
    './dataset.html',
    fullscreen=True,
    node_metrics=['louvain'],
    node_color='louvain',
    node_size_range=(3, 15),
    max_categorical_colors=30,
    default_edge_type='curve',
    node_border_color_from='node',
    default_node_label_size=14,
    node_size=G.degree
)


In [ ]:
from IPython.display import IFrame
IFrame('graph.html', width=800, height=600)